In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds

C:\Users\Csanad\AppData\Roaming\Python\Python39\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Data aquisition&preparation

In [ ]:
# Oxford-IIIT pet dataset:
# https://github.com/tensorflow/datasets/tree/master/tensorflow_datasets/datasets/oxford_iiit_pet
dataset_name = "oxford_iiit_pet"
split = [
    "train[:60%]+test[:60%]",
    "train[60%:80%]+test[60%:80%]",
    "train[80%:]+test[80%:]"
]

In [15]:
# Oxford Flowers 102 dataset:
# https://github.com/tensorflow/datasets/tree/master/tensorflow_datasets/datasets/oxford_flowers102
dataset_name = "oxford_flowers102"
split = [
    "train[:60%]+validation[:60%]+test[:60%]",
    "train[60%:80%]+validation[60%:80%]+test[60%:80%]",
    "train[80%:]+validation[80%:]+test[80%:]"
]

In [12]:
# Hyperparameters
img_size = 64
batch_size = 64

In [3]:
def preprocess_image(data):
    height = tf.shape(data["image"])[0]
    width = tf.shape(data["image"])[1]
    crop_size = tf.minimum(height, width)
    img = tf.image.crop_to_bounding_box(
        data["image"],
        (height - crop_size) // 2,
        (width - crop_size) // 2,
        crop_size,
        crop_size,
    )
    img = tf.cast(img, dtype=tf.float32)
    img = tf.image.resize(img, size=(img_size, img_size), antialias=True)
    return tf.clip_by_value(img / 255.0, 0.0, 1.0)

In [16]:
def preprocess_dataset(dataset):
    return (dataset.map(preprocess_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)
            .batch(batch_size, drop_remainder=True)
            .shuffle(2 * batch_size)
            .prefetch(buffer_size=tf.data.AUTOTUNE))

def load_data(dataset_name):
    train_ds, val_ds, test_ds = tfds.load(dataset_name, split=split, shuffle_files=True)
    train_ds = preprocess_dataset(train_ds)
    val_ds = preprocess_dataset(val_ds)
    test_ds = preprocess_dataset(test_ds)
    return train_ds, val_ds, test_ds

In [18]:
train_ds, val_ds, test_ds = load_data(dataset_name)